In [1]:
!pip install pandas
!pip install matplotlib
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 1.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 59.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 47.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.5/102.5 kB 1.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 32.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.0/325.0 kB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 46.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 104.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgr

In [4]:
!pip install datasets evaluate tensorboard transformers accelerate kagglehub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 1.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:
import torch
import pandas as pd
import numpy as np
import re
from matplotlib import pylab as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from transformers import AutoTokenizer, Trainer, TrainingArguments, AutoModelForSequenceClassification

from datasets import Dataset
from datasets import concatenate_datasets
from datasets import DatasetDict
from datasets import ClassLabel

import os
os.environ["KAGGLE_USERNAME"] = "USERNAME"
os.environ["KAGGLE_KEY"] = "KAGGLE_KEY"
import kagglehub

In [6]:
# Download the latest version.
dataset_path = kagglehub.dataset_download("anonymous/wolbanking77v1")
print(dataset_path)

100%|██████████| 1.02M/1.02M [00:00<00:00, 1.29MB/s]

Extracting files...
/root/.cache/kagglehub/datasets/anonymous/wolbanking77v1/versions/4


In [9]:
!ls /root/.cache/kagglehub/datasets/anonymous/wolbanking77v1/versions/4/wolbanking77v1/full

test  train


In [2]:
# Set the Seed for Reproducible Results
seed = 42
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [3]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [4]:
def compute_metrics(y_true, y_pred):
    print("f1_score:", metrics.f1_score(y_true, y_pred, average='weighted'))
    print("precision_score:", metrics.precision_score(y_true, y_pred, average='weighted'))
    print("recall_score:", metrics.recall_score(y_true, y_pred, average='weighted'))

In [5]:
df_train = pd.read_csv(
    "/root/.cache/kagglehub/datasets/anonymous/wolbanking77v1/versions/4/wolbanking77v1/full/train/train.csv")

df_test = pd.read_csv(
    "/root/.cache/kagglehub/datasets/anonymous/wolbanking77v1/versions/4/wolbanking77v1/full/test/test.csv")

df_train = df_train[['input_wo', 'label']]
df_test = df_test[['input_wo', 'label']]

#removal of punctuation marks
df_train['input_wo_clean'] = df_train['input_wo'].replace(regex=True, to_replace = '[^\\w\\s]', value = ' ')
df_test['input_wo_clean'] = df_test['input_wo'].replace(regex=True, to_replace = '[^\\w\\s]', value = ' ')

# Apply lowercase
df_train['input_wo_clean'] = df_train['input_wo_clean'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df_test['input_wo_clean'] = df_test['input_wo_clean'].apply(lambda x: " ".join(x.lower() for x in x.split()))

train_ds = Dataset.from_pandas(df_train[['input_wo_clean', 'label']], split="train")
test_ds = Dataset.from_pandas(df_test[['input_wo_clean', 'label']], split="test")

raw_dataset = DatasetDict({"train": train_ds, "test": test_ds})
# raw_dataset = raw_dataset.remove_columns("__index_level_0__")
raw_dataset =  raw_dataset.rename_column("input_wo_clean", "text") # to match Trainer
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 7832
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1959
    })
})

In [6]:
# Tokenize helper function
def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True, return_tensors="pt")

In [7]:
from transformers import AutoTokenizer

# Model id to load the tokenizer
model_id = "bert-base-uncased"
# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [8]:
tokenized_dataset = raw_dataset.map(tokenize, batched=True,remove_columns=["text"])

Map:   0%|          | 0/7832 [00:00<?, ? examples/s]

Map:   0%|          | 0/1959 [00:00<?, ? examples/s]

In [9]:
labels = []
for label in tokenized_dataset["train"]:
    labels.append(label['label'])
labels = list(set(labels))
len(labels)

77

In [10]:
new_features = tokenized_dataset['train'].features.copy()
new_features['label'] = ClassLabel(names=list(set(labels)))
tokenized_dataset['train'] = tokenized_dataset['train'].cast(new_features)
new_features = tokenized_dataset['test'].features.copy()
new_features['label'] = ClassLabel(names=list(set(labels)))
tokenized_dataset['test'] = tokenized_dataset['test'].cast(new_features)

Casting the dataset:   0%|          | 0/7832 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1959 [00:00<?, ? examples/s]

In [11]:
from transformers import AutoModelForSequenceClassification

# Prepare model labels - useful for inference
num_labels = len(labels)
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

# Download the model from huggingface.co/models
model = AutoModelForSequenceClassification.from_pretrained(
    model_id, num_labels=num_labels, label2id=label2id, id2label=id2label)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
import evaluate
import numpy as np

# Metric Id
metric = evaluate.load("f1")

# Metric helper method
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels, average="weighted")

In [17]:
#del trainer
#torch.cuda.empty_cache()

In [13]:
from huggingface_hub import HfFolder
from transformers import Trainer, TrainingArguments

# Id for remote repository
repository_id = "BERT-base-banking77-full-wolof"

# Define training args
training_args = TrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=16, # 8 16 32
    per_device_eval_batch_size=8, # 8
    learning_rate=2e-05,
	num_train_epochs=20, # 20
    warmup_ratio=0.1,
    weight_decay=0.01,
		# PyTorch 2.0 specifics 
    # bf16=True, # bfloat16 training 
	# torch_compile=True, # optimizations
    optim="adamw_torch_fused", # improved optimizer 
    # logging & evaluation strategies
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=200,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    # push to hub parameters
    report_to="tensorboard",
    push_to_hub=False,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
    remove_unused_columns=False

)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

In [14]:
# Start training
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,4.327100,4.245815,0.018582
2,4.083000,3.650564,0.127002
3,3.206700,3.053345,0.308455
4,2.742000,2.677301,0.416152
5,2.301000,2.468802,0.493443
6,2.049400,2.338958,0.526382
7,1.784500,2.305984,0.535808
8,1.628100,2.258097,0.550902
9,1.437200,2.246423,0.556377
10,1.285800,2.260932,0.566067


TrainOutput(global_step=9800, training_loss=1.6124656513759068, metrics={'train_runtime': 8020.9368, 'train_samples_per_second': 19.529, 'train_steps_per_second': 1.222, 'total_flos': 4.12414688108544e+16, 'train_loss': 1.6124656513759068, 'epoch': 20.0})

In [15]:
trainer.evaluate(tokenized_dataset["test"])

{'eval_loss': 2.260932445526123,
 'eval_f1': 0.5660667164521239,
 'eval_runtime': 29.6124,
 'eval_samples_per_second': 66.155,
 'eval_steps_per_second': 8.274,
 'epoch': 20.0}

In [16]:
# Save processor and create model card
tokenizer.save_pretrained(repository_id)

('BERT-base-banking77-full-wolof/tokenizer_config.json',
 'BERT-base-banking77-full-wolof/special_tokens_map.json',
 'BERT-base-banking77-full-wolof/vocab.txt',
 'BERT-base-banking77-full-wolof/added_tokens.json',
 'BERT-base-banking77-full-wolof/tokenizer.json')

In [17]:
model_id = "/workspace/NLP_TASKS/BERT-base-banking77-full-wolof/checkpoint-9800"
tokenizer = AutoTokenizer.from_pretrained(repository_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id, use_safetensors=True).to("cuda")

In [18]:
def predict(batch, model):
    input_ids = torch.tensor(batch['input_ids']).to(device).unsqueeze(0)
    token_type_ids = torch.tensor(batch['token_type_ids']).to(device).unsqueeze(0)
    attention_mask = torch.tensor(batch['attention_mask']).to(device).unsqueeze(0)
    with torch.no_grad():
        logits = model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask).logits
        predicted = torch.argmax(logits, dim=-1)[0]
    return predicted

In [19]:
model.to("cuda")
predictions = [predict(batch, model) for batch in tokenized_dataset["test"]]

In [20]:
predictions[0]

tensor(8, device='cuda:0')

In [21]:
y_pred = []
for y in predictions:
    y_pred.append(labels[y])
y_pred

['Refund not showing up',
 'transfer into account',
 'card payment fee charged',
 'direct debit payment not recognised',
 'getting spare card',
 'pending card payment',
 'request refund',
 'receiving money',
 'top up by card charge',
 'why verify identity',
 'cash withdrawal not recognised',
 'activate my card',
 'declined transfer',
 'transaction charged twice',
 'card payment fee charged',
 'exchange charge',
 'pending transfer',
 'card acceptance',
 'declined card payment',
 'direct debit payment not recognised',
 'pending top up',
 'top up failed',
 'pending cash withdrawal',
 'exchange rate',
 'apple pay or google pay',
 'balance not updated after cheque or cash deposit',
 'direct debit payment not recognised',
 'pending cash withdrawal',
 'supported cards and currencies',
 'cash withdrawal charge',
 'getting virtual card',
 'card about to expire',
 'cancel transfer',
 'apple pay or google pay',
 'declined transfer',
 'request refund',
 'declined cash withdrawal',
 'pending cash w

In [22]:
y_test = []
for y in tokenized_dataset["test"]['label']:
    y_test.append(labels[y])
y_test

['activate my card',
 'transfer into account',
 'card payment fee charged',
 'unable to verify identity',
 'getting spare card',
 'pending card payment',
 'request refund',
 'receiving money',
 'top up by card charge',
 'why verify identity',
 'cash withdrawal not recognised',
 'visa or mastercard',
 'card not working',
 'transaction charged twice',
 'card payment fee charged',
 'exchange via app',
 'getting spare card',
 'card acceptance',
 'lost or stolen card',
 'request refund',
 'pending top up',
 'topping up by card',
 'pending cash withdrawal',
 'declined card payment',
 'apple pay or google pay',
 'top up failed',
 'direct debit payment not recognised',
 'pending cash withdrawal',
 'supported cards and currencies',
 'cash withdrawal charge',
 'virtual card not working',
 'card about to expire',
 'cancel transfer',
 'apple pay or google pay',
 'declined transfer',
 'verify my identity',
 'card not working',
 'balance not updated after cheque or cash deposit',
 'extra charge on s

In [23]:
# from sklearn import metrics
# y_test = test_df['label']
print(metrics.classification_report(y_test, y_pred))

                                                  precision    recall  f1-score   support

                           Refund not showing up       0.46      0.55      0.50        33
                                activate my card       0.62      0.47      0.54        32
                                       age limit       0.64      0.84      0.73        19
                         apple pay or google pay       0.71      0.68      0.69        25
                                     atm support       0.52      0.80      0.63        15
                                automatic top up       0.65      0.62      0.64        24
         balance not updated after bank transfer       0.52      0.50      0.51        34
balance not updated after cheque or cash deposit       0.62      0.64      0.63        36
                         beneficiary not allowed       0.44      0.52      0.48        31
                                 cancel transfer       0.69      0.80      0.74        30
         

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [24]:
metrics.f1_score(y_test, y_pred, average='weighted')

0.5508718886671862

In [25]:
metrics.accuracy_score(y_test, y_pred)

0.554364471669219

In [26]:
metrics.precision_score(y_test, y_pred, average='weighted')

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


0.5584550861624186

In [27]:
metrics.recall_score(y_test, y_pred, average='weighted')

0.554364471669219

In [28]:
def compute_metrics(y_true, y_pred):
    print("f1_score:", metrics.f1_score(y_true, y_pred, average='weighted'))
    print("precision_score:", metrics.precision_score(y_true, y_pred, average='weighted'))
    print("recall_score:", metrics.recall_score(y_true, y_pred, average='weighted'))

In [29]:
compute_metrics(y_test, y_pred)

f1_score: 0.5508718886671862
precision_score: 0.5584550861624186
recall_score: 0.554364471669219


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
